In [1]:
using JuMP, Gurobi
using CSV, LinearAlgebra, DataFrames
using Plots
using DelimitedFiles

In [2]:
#load social and ecological data and orgnize 
FL = repeat([Float64], inner=15)
dtype = append!([String], FL);

regional_EF = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/new_EF_SC1.csv",header=1,delim=",", types=dtype) |> DataFrame    
capacity = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/capacity2.csv",header=1,delim=",", types=dtype) |> DataFrame    
distance = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/distance.csv",header=1,delim=",") |> DataFrame 
LCA_model = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/LCA_model2.csv",header=1,delim=",") |> DataFrame 
D_Dsoc = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/D_Dsoc1.csv",header=1,delim=",") |> DataFrame
GDP = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/GDP.csv",header=1,delim=",") |> DataFrame;
emi_sink = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/SC_regional/emission_sink1.csv",header=1,delim=",") |> DataFrame;

In [3]:
cell_demand = 0.001*164.98*(1.369*1e6)*2           # annual demand of Li battery for tesla (1.369M EV/yr, ~2 NMC111 pack/EV, 164.98 kg/pack (35kwh/pack), 80~100 kWh per EV)

global_sink = 1.099e10                        # global pub (ocean) CO2 sequestration (ton/yr)
global_sink_tot = 2.236e10                  # global total (ocean+land) CO2 sequestration (ton/yr)
global_emi = 3.53e10                          # global CO2 emission (ton/yr)
global_gdp = 96882e9                          # 2021 global GDP ($/yr)
es_ratio = global_sink/global_emi
es_ratio_tot = global_sink_tot/global_emi
emission_c = emi_sink[!, "emission"]          # national CO2 emission (ton/yr)
sink_c = emi_sink[!, "sink ton/yr"]           # national CO2 sink (ton/yr)

# D = D_Dsoc[!, "D"]          # national CO2 emission (ton/yr)
Dsoc = D_Dsoc[!, "Dsoc ton/yr"]           # national CO2 sink (ton/yr)

EF_trans = 1.005/10000                        # ton CO2/km*ton (The average freight truck in the U.S. emits 161.8 grams of CO2 per ton-mile)
process = LCA_model[!,"process"]
countries = capacity[!,"country"]
ncty = size(countries,1)                          # No. of countries
nproc = size(process,1);                          # No. of processes 

mkt_loc = findfirst(isequal("United States"), countries)
mkt_proc = findfirst(isequal("battery"), process)

# seperate model
cathode = collect(1:4)
cell = collect(5:10)
noncell = [12,13]
battery = [11,14]
scaler = LCA_model[!,"scaler"]
price = LCA_model[!,"price (usd/ton product)"]
vGDP = GDP[!,"GDP usd"];

In [4]:
# penalty = 10000 # ton/yr
penalty = 5; # ton/yr

In [5]:
up_cath = scaler[1:4] * scaler[5] * scaler[11]
up_cell = scaler[5:10] * scaler[11]
cell_sef = scaler[11]
up_noncell = scaler[12:13] * scaler[14]
noncell_sef = scaler[14]
battery_sef = scaler[15];

In [54]:
ipt = []
ipt = vcat(up_cath, up_cell, cell_sef, up_noncell, noncell_sef, battery_sef) .* cell_demand
input_amount = ipt

15-element Vector{Float64}:
  49670.291589732
  69512.470736544
  67696.846500888
  69382.78329114
 129687.44540400001
  72376.074329
   5239.896784
   1309.974196
  31111.887154999997
  60258.81301599999
 327493.549
   2258.5762
   1806.86096
 451715.24
 451715.24

In [61]:
Efficiency = [0.5, 0.7, 0.7, 0.8, 0.8, 0.8, 0.9, 1]

8-element Vector{Float64}:
 0.5
 0.7
 0.7
 0.8
 0.8
 0.8
 0.9
 1.0

---

### Solve subproblem to get dual $\pi$ and $\alpha$

In [66]:
function reformu_pi(eff, col_idx, proc_idx, ls_cstr)
    pi_matrix = zeros(ncty, nproc)
    pi = [getdual(con) for con in ls_cstr]
    B = []
    for k in proc_idx
        append!(B, eff * scaler[k] * ones(ncty))
    end
    pi = pi .* B
    pi = reshape(pi, ncty, size(proc_idx)[1])
    pi_matrix[:,col_idx] = sum(pi, dims=2)
    
    return pi_matrix
end

reformu_pi (generic function with 2 methods)

In [67]:
function subprob(x_hat, eff)
    model = Model(Gurobi.Optimizer)
    set_silent(model)
    @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
    @variable(model, s >= 0)

    # node output flow constraint
    cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == eff*x_hat[i,k]) for k in 1:nproc for i in 1:ncty]

    # cathode LCA constraints (index=5)
    cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == eff*x_hat[j,5]*scaler[k]) for k in cathode for j in 1:ncty]

    # cell LCA constraints (index=11)
    cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == eff*x_hat[j,11]*scaler[k]) for k in cell for j in 1:ncty]

    # non cell LCA constraints (index=14)
    cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == eff*x_hat[j,14]*scaler[k]) for k in noncell for j in 1:ncty]

    # battery LCA constraints (index=15)
    cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == eff*x_hat[j,15]*scaler[k]) for k in battery for j in 1:ncty]
    
    @constraint(model, cstr_alp, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) + s >= cell_demand)
    for j in 1:ncty-1
        @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0)
    end 
    
    
    #######################
    tranD = Vector{AffExpr}(undef, ncty)
    tranS= Vector{AffExpr}(undef, ncty)
    for j in 1:ncty
        arc_emi = 0
        for i in 1:ncty
            amount = sum(y[i,j,k] for k in 1:nproc)
            arc_emi += (amount * distance[!, 2:end][i,j] * EF_trans)
        end
        tranD[j] = arc_emi
        tranS[j] = arc_emi * (sink_c[j]/emission_c[j] + es_ratio)
    end
    
    
    @objective(model, Min, sum(tranD-tranS) + s*penalty)
    JuMP.optimize!(model)
    
    qy_hat = JuMP.objective_value(model)
    sub_y = JuMP.value.(y)
    sub_s = JuMP.value.(s)
    
    
    ####################### 
    op_pi = [getdual(con) for con in cstr_op]
    op_pi = reshape(op_pi, ncty, nproc) * eff

    cth_pi = reformu_pi(eff, 5, cathode, cstr_cth)
    cell_pi = reformu_pi(eff, 11, cell, cstr_cell)
    noncell_pi = reformu_pi(eff, 14, noncell, cstr_noncell)
    battery_pi = reformu_pi(eff, 15, battery, cstr_battery)
    pi_matrix = op_pi + cth_pi + cell_pi + noncell_pi + battery_pi
    
    alp = getdual(cstr_alp)

    
    #######################
    result = Dict(["qyhat"=>qy_hat, "y_opt"=>sub_y, "s_opt"=>sub_s, "pi"=>pi_matrix, "alp"=>alp]);
    return result
    
end


subprob (generic function with 1 method)

### Solve master problem

In [68]:
M = -5e7

-5.0e7

In [69]:
function masterprob(cuts)
    model = Model(Gurobi.Optimizer)
    set_silent(model)
    @variable(model, x[1:ncty, 1:nproc] >= 0)
    @variable(model, theta >= M)
    
    for k in 1:nproc
        for i in 1:ncty
            @constraint(model, x[i,k] <= capacity[!, 2:end][i,k])
        end
    end
#     for k in 1:nproc
#         @constraint(model, sum(x[i,k] for i in 1:ncty) == input_amount[k])
#     end
    
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
    proM = x * price

    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)*ones(nproc,1) 
    
    if cuts != []
        for cut in cuts
            G = cut["gradient"]
            g = cut["intersection"]
            @constraint(model, sum(x .* G) + theta >= g)
        end
    end
    
    @objective(model, Min, sum(proD-proS)+theta)
    JuMP.optimize!(model);
    
    x_hat = JuMP.value.(x)
    theta_hat = JuMP.value.(theta)
    z_lb = JuMP.objective_value(model)
    
    result = Dict(["x_hat"=>x_hat, "z_lb"=>z_lb])
    return result
end

masterprob (generic function with 1 method)

### Generate cuts

In [75]:
function add_cuts(x, Efficiency)
    G = [] # gradient
    g = 0 # intersection
    qy = 0
    
    for eff in Efficiency
        res = subprob(x, eff)
        gradient = -1 * res["pi"]
        
        push!(G, gradient)
        g += cell_demand * res["alp"]
        qy += res["qyhat"]
    end
    
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)*ones(nproc,1) 
    z_hat = sum(proD - proS) + qy/size(D)[1]
    Exp_G = sum(G) / size(D)[1]
    Exp_g = sum(g) / size(D)[1]
    
    cut = Dict(["gradient"=>Exp_G, "intersection"=>Exp_g])
    return cut, z_hat
end


add_cuts (generic function with 1 method)

### L Shape Method

In [76]:
function main(toler)
    i = 0
    z_ub = 1e7
    x_opt = 0
    
    cuts = []
    res0 = masterprob(cuts)
    x_hat = res0["x_hat"]
    z_lb = res0["z_lb"]
    
    while (z_ub - z_lb) > toler * min(abs(z_ub), abs(z_lb))
        new_cut, z_hat = add_cuts(x_hat, Efficiency)
        if z_hat < z_ub
            z_ub = z_hat
            x_opt = x_hat
        end
        
        push!(cuts, new_cut)
        
        res1 = masterprob(cuts)
        z_lb = res1["z_lb"]
        x_hat = res1["x_hat"]
        
        i += 1
    end
    
    result = Dict(["z_lb"=>z_lb, "z_ub"=>z_ub, "x_opt"=>x_opt, "iteration"=>i])
    return result
#     println(z_lb, z_ub, x_opt, i)
    
end

main (generic function with 1 method)

In [79]:
result = main(0.05)

Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only -

Dict{String, Any} with 4 entries:
  "z_lb"      => -8.07419e6
  "x_opt"     => [6200.0 0.0 … 0.0 0.0; 1770.29 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.…
  "iteration" => 4
  "z_ub"      => -7.79699e6

In [77]:
i = 0
z_ub = 1e7
x_opt = 0

cuts = []
res0 = masterprob(cuts)
x_hat = res0["x_hat"]
z_lb = res0["z_lb"]

Academic license - for non-commercial use only - expires 2023-11-27


-5.838412704242925e7

In [83]:
sum(result["x_opt"], dims=1)

1×15 Matrix{Float64}:
 49670.3  69512.5  67696.8  69382.8  …  1806.86  4.51715e5  4.51715e5

In [81]:
Efficiency

8-element Vector{Float64}:
 0.5
 0.7
 0.7
 0.8
 0.8
 0.8
 0.9
 1.0

In [28]:
# result = main(0.005)

In [26]:
# res_x = DataFrame(xopt, :auto)
# rename!(res_x, ["x$i" => proc for (i, proc) in enumerate(process)])
# insertcols!(res_x, 1, :country => countries)

In [261]:
# CSV.write("/Users/bourg/Desktop/Oct31/eco_stoc.csv", res_x) 

"/Users/bourg/Desktop/Oct31/eco_stoc.csv"

---

In [18]:
function det_model()
    model = Model(Gurobi.Optimizer)
    @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
    @variable(model, x[1:ncty, 1:nproc] >= 0)
    
    cstr_cap = [@constraint(model, x[i,k] <= capacity[!, 2:end][i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_ip = [@constraint(model, sum(x[i,k] for i in 1:ncty) == input_amount[k]) for k in 1:nproc]
    
    cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == x[i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
    cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,11]*scaler[k]) for k in cell for j in 1:ncty]
    cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
    cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,15]*scaler[k]) for k in battery for j in 1:ncty]
    @constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) == cell_demand)
    for j in 1:ncty-1
        @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0)
    end
    
    ############################
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
    proM = x * price

    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)*ones(nproc,1) 
    
    ###
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
    proM = x * price

    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)*ones(nproc,1) 
    #######
    
    tranD = Vector{AffExpr}(undef, ncty)
    tranS= Vector{AffExpr}(undef, ncty)
    for j in 1:ncty
        arc_emi = 0
        for i in 1:ncty
            amount = sum(y[i,j,k] for k in 1:nproc)
            arc_emi += (amount * distance[!, 2:end][i,j] * EF_trans)
        end
        tranD[j] = arc_emi
        tranS[j] = arc_emi * (sink_c[j]/emission_c[j] + es_ratio)
    end

    Allo_soc = (proD+tranD) ./ emission_c .* Dsoc

    vec_S = proS + tranS
    vec_D = proD + tranD
    
    @objective(model, Min, sum(vec_D-vec_S))
    JuMP.optimize!(model)
    
    opt_x = JuMP.value.(x)
    opt_y = JuMP.value.(y)
    opt_obj = JuMP.objective_value(model)
    
    result = Dict(["opt_x"=>opt_x, "opt_obj"=>opt_obj, "opt_y"=>opt_y])
    
    return result
    
end

det_model (generic function with 1 method)

In [19]:
det_res = det_model()

Academic license - for non-commercial use only - expires 2023-11-27
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1365 rows, 13950 columns and 28770 nonzeros
Model fingerprint: 0x7de78d9d
Coefficient statistics:
  Matrix range     [4e-03, 1e+00]
  Objective range  [7e-03, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+11]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 1119 rows and 12768 columns
Presolve time: 0.01s
Presolved: 246 rows, 1182 columns, 2548 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.0366234e+07   3.023558e+05   0.000000e+00      0s
      77   -8.1875518e+06   0.000000e+00   0.000000e+00      0s

Solved in 77 iterations and 0.01 seconds
Optimal objective -8.187551773e+06

User-callback calls 121, time in user-callback 0.

Dict{String, Any} with 3 entries:
  "opt_y"   => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.…
  "opt_obj" => -8.18755e6
  "opt_x"   => [6200.0 0.0 … 0.0 0.0; 1770.29 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 …

In [21]:
optx_det = det_res["opt_x"]
opty_det = det_res["opt_y"]
opt_obj = det_res["opt_obj"]

-8.187551773144178e6

In [27]:
optx_det

30×15 Matrix{Float64}:
  6200.0       0.0      0.0      0.0   …     0.0   0.0        0.0
  1770.29      0.0      0.0      0.0         0.0   0.0        0.0
  2200.0       0.0  67696.8  67000.0         0.0   0.0        0.0
   500.0    3100.0      0.0   2382.78        0.0   0.0        0.0
 39000.0       0.0      0.0      0.0         0.0   4.51715e5  0.0
     0.0       0.0      0.0      0.0   …     0.0   0.0        0.0
     0.0       0.0      0.0      0.0         0.0   0.0        0.0
     0.0   66412.5      0.0      0.0         0.0   0.0        0.0
     0.0       0.0      0.0      0.0         0.0   0.0        0.0
     0.0       0.0      0.0      0.0         0.0   0.0        0.0
     0.0       0.0      0.0      0.0   …     0.0   0.0        0.0
     0.0       0.0      0.0      0.0         0.0   0.0        0.0
     0.0       0.0      0.0      0.0         0.0   0.0        0.0
     ⋮                                 ⋱                      
     0.0       0.0      0.0      0.0         0.0   0.0  

In [ ]:
res_x = DataFrame(optx_det, :auto)
rename!(res_x, ["x$i" => proc for (i, proc) in enumerate(process)])
insertcols!(res_x, 1, :country => countries)